# Bias Detection

**Main Objective:**  
Detect errors and biases introduced by the CV parser when extracting skills from raw CV text.
We perform this inspection using both rule based (regular expressions) and semantic techniques.

1. **Error detection steps:**  
    - Identify errors in candidates **Driving Licenses** and **Language Skills** using Regex.  
    - Uncover errors in candidates **Job Experience** using exact matching and semantic approach.  

2. **Bias detection:**
    - Analyze the errors identified in Step 1 for the groups previously examined (see `distribution_analysis.ipynb`) to determine whether the parser has disadvantaged or advantaged any of them.

In [ ]:
%load_ext autoreload 
%autoreload 2

import json
import os

import numpy as np
import polars as pl
from huggingface_hub import login
from Levenshtein import ratio

from hiring_cv_bias.bias_detection.fuzzy.matcher import SemanticMatcher
from hiring_cv_bias.bias_detection.fuzzy.parser import JobParser
from hiring_cv_bias.bias_detection.rule_based.data import (
    add_demographic_info,
)
from hiring_cv_bias.bias_detection.rule_based.evaluation.compare_parser import (
    compute_candidate_coverage,
)
from hiring_cv_bias.bias_detection.rule_based.extractors import (
    extract_driver_license,
    extract_languages,
    norm_driver_license,
    norm_languages,
)
from hiring_cv_bias.bias_detection.rule_based.patterns import (
    driver_license_pattern_eng,
    jobs_pattern,
    languages_pattern_eng,
    normalized_jobs,
)
from hiring_cv_bias.bias_detection.rule_based.utils import (
    print_highlighted_cv,
    print_report,
)
from hiring_cv_bias.bias_detection.utils import extract_skill_cases
from hiring_cv_bias.config import (
    CANDIDATE_CVS_TRANSLATED_CLEANED_PATH,
    CLEANED_REVERSE_MATCHING_PATH,
    CLEANED_SKILLS,
    DRIVING_LICENSE_FALSE_NEGATIVES_PATH,
    GENDER_JOBS_PATH,
    JOB_TITLE_FALSE_NEGATIVES_PATH,
    LANGUAGE_SKILL_FALSE_NEGATIVES_PATH,
)
from hiring_cv_bias.utils import load_data

pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(200);

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "True"
with open("token.json", "r") as token:
    login(token=json.load(token)["token"])

!python -m spacy download en_core_web_sm 

### Load the data 

In [ ]:
df_cv_raw = load_data(CANDIDATE_CVS_TRANSLATED_CLEANED_PATH)
df_skills = load_data(CLEANED_SKILLS)
df_info_candidates = load_data(CLEANED_REVERSE_MATCHING_PATH)

In [ ]:
df_info_candidates = df_info_candidates.with_columns(
    pl.when(pl.col("LATITUDE") > 44.5)
    .then(pl.lit("NORTH"))
    .when(pl.col("LATITUDE") < 42)
    .then(pl.lit("SOUTH"))
    .otherwise(pl.lit("CENTER"))
    .alias("Location")
)

df_cv_raw = df_cv_raw.with_columns(
    pl.when(pl.col("len_anon") < 1000)
    .then(pl.lit("SHORT"))
    .when(pl.col("len_anon") < 2500)
    .then(pl.lit("MEDIUM"))
    .otherwise(pl.lit("LONG"))
    .alias("length")
)

### Bias detection for Driver Licences

- **Pre-processing step –> driving licence flag**  
  We call `add_demographic_info()` to add a Boolean column, `has_driving_license`, to the CV dataframe. This flag will help us compare what the regex detects in the raw CV text with what the parser extracted as driving licence for each candidate, allowing us to identify potential omissions in the parsing step.

- **How the flag is generated**  
  - A single case insensitive regex (`driver_license_pattern_eng`) looks for common phrases such as “driving license B”, "C1 driving licence” or even “own car”.  
  - The helper function `extract_driver_license(text)` returns `True` if the regex matches anywhere in the CV text.  

- **Resulting columns in `df_cv`**  
  -  Same as before
  - `Gender`, `Location` —> from the candidates sheet
  - `has_driving_license` —> `True` if any licence is mentioned, otherwise `False`

- **Note**  
  For now we only care whether a candidate has *any* licence. (given that the driver license type column contains a handful of null values (see `data_cleaning.ipynb`))
  
  The same regex already captures specific categories (A, B, C…), so the analysis could be extended later if we want to explore potential biases tied to particular licence types.

In [ ]:
df_cv = add_demographic_info(df_cv_raw, df_info_candidates)
df_cv.head()

**Comparing parser output and regex detection**

The `compute_candidate_coverage()` function evaluates how well the parsing system detects a specific category of skills  by comparing it to our approach. 

In this case, the chosen category is  `"DRIVERSLIC"` and we use a custom regex based extractor applied directly to the raw CV text.


This step is crucial for measuring the parser’s coverage by quantifying **false negatives**. (skills that are mentioned in the CV but missed by the parser)

**Output breakdown:** 
   1. **Regex positive candidates**: number of unique candidates flagged by our rule based extractor.
   2. **Parser positive unique candidates**: number of unique candidates flagged by the parser.

- **Both regex & parser**: candidates detected by both methods.
- **Only regex**: candidates our regex caught but the parser missed. 
- **Only parser**: candidates the parser flagged but our regex did not.

Then `print_report()` displays:
   - The overall confusion matrix and derived metrics (accuracy, precision, recall, F1).

In [ ]:
res_dl = compute_candidate_coverage(
    df_cv=df_cv,
    df_parser=df_skills,
    skill_type="DRIVERSLIC",
    extractor=extract_driver_license,
    norm=norm_driver_license,
)

print("Confusion matrix:", res_dl.conf)

The parser achieves **high precision (~83 %)** but **low recall (~41 %)**.  
In other words, when it flags a skill it is usually correct, yet it **misses more than half of the skills** that the regex finds.

Let's dive into **false negatives (FN)**  
   * We’ll highlight in **red** the exact terms captured by the regex directly inside the CV text, making it easy to verify their presence at a glance.

In [ ]:
df_fn = pl.DataFrame(res_dl.fn_rows)
sample = df_fn.sample(n=2, shuffle=True)
for row in sample.to_dicts():
    print_highlighted_cv(row, pattern=driver_license_pattern_eng)

In [ ]:
print(f"False negatives matching snippet pattern: {df_fn.height}")
df_fn.write_csv(DRIVING_LICENSE_FALSE_NEGATIVES_PATH, separator=";")
print("Saved filtered false negatives!")

In [ ]:
%%bash --bg 
cd ..

# for Unix users  
.venv/bin/python -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

# for Windows users
#.venv/Scripts/python.exe -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

### Bias Detection Metrics

To assess the model’s fairness, we employed the following **bias detection metrics**:

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **Equality of Opportunity&nbsp;(TPR parity)** | $$\text{TPR}_g = \frac{TP_g}{TP_g + FN_g} $$ | $\text{TPR}_g$ equal for every $g$ ensures that **every individual who truly qualifies** for a positive outcome has the **same chance** of being correctly identified, regardless of group membership. |
| <br> **Calibration&nbsp;\(NPV\)** | $$\text{NPV}_g = \frac{TN_g}{TN_g + FN_g}\qquad $$ | $\text{NPV}_g$ parity for every $g$ ensures that **when the model predicts a negative outcome**, the probability of being correct is the **same** for every group. |
| <br> **Selection Rate** | $$\text{SR}_g = \frac{TP_g + FP_g}{TP_g + FP_g + TN_g + FN_g} $$ | Share of individuals in group $g$ predicted positive (selected). |
| <br> **Disparate Impact (DI)** | $$\displaystyle DI = \frac{\text{SR}_{\text{target}}}{\text{SR}_{\text{reference}}}$$ | Ratio of selection rates; values **\< 0.80** (four-fifths rule) indicate potential adverse impact against the target group. |



<u>All these metrics were computed for the **Gender** and **Location** groups to detect and quantify possible bias in the selection process.</u>


In [ ]:
print_report(
    result=res_dl,
    df_population=df_cv,
    reference_col="Male",
    group_col="Gender",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_dl,
    df_population=df_cv,
    reference_col="NORTH",
    group_col="Location",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_dl,
    df_population=df_cv,
    reference_col="LONG",
    group_col="length",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

### Bias detection for Language Skill

Just as we applied a simple presence check for driving licenses, we handle languages with a more granular, ad-hoc normalization and extraction pipeline that recognizes each specific language individually rather than merely flagging “has any language”.

Main **steps** for doing this are:

* Build a reverse lookup map (`_reverse_language_map`) by iterating over each language code in `LANGUAGE_VARIANTS` (populated from pycountry with English name variants `alpha_2`) and all its known name variants, storing entries like "english" -> "en", "italian" -> "it", etc.
* Apply `norm_languages`to every extracted mention from our regex based extractor so that each occurrence (like "English B2” is mapped to a clean ISO code ("en").
* Once all language mentions have been normalized to ISO codes via `norm_languages`, we invoke the coverage routine to quantify how well the parser matches our “ground truth” extractions.

In [ ]:
res_lg = compute_candidate_coverage(
    df_cv=df_cv,
    df_parser=df_skills,
    skill_type="Language_Skill",
    extractor=extract_languages,
    norm=norm_languages,
)

print("Confusion matrix:", res_lg.conf)

In [ ]:
df_fn = pl.DataFrame(res_lg.fn_rows)
sample = df_fn.sample(n=2, shuffle=True)
for row in sample.to_dicts():
    print_highlighted_cv(row, pattern=languages_pattern_eng)

In [ ]:
print(f"False negatives matching snippet pattern: {df_fn.height}")
df_fn.write_csv(LANGUAGE_SKILL_FALSE_NEGATIVES_PATH, separator=";")
print("Saved filtered false negatives to false_negative.csv")

In [ ]:
%%bash --bg 
cd ..
# for Unix users
.venv/bin/python -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

# for Windows users
#.venv/Scripts/python.exe -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

In [ ]:
print_report(
    result=res_lg,
    df_population=df_cv,
    reference_col="Male",
    group_col="Gender",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_lg,
    df_population=df_cv,
    reference_col="NORTH",
    group_col="Location",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_lg,
    df_population=df_cv,
    reference_col="LONG",
    group_col="length",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

### Bias detection for Job Title

Loading list of jobs from [ESCO](https://esco.ec.europa.eu/en/about-esco) and filtering out those that are too specific (length > 3).

In [ ]:
df_skills_cleaned = df_skills.with_columns(
    pl.col("Skill")
    .str.to_lowercase()
    .str.replace_all("(m/f)", "", literal=True)
    .str.strip_chars()
    .alias("Skill")
)

The **bias detection pipeline for job titles** consists of two main components:

* **JobParser**: a class that extracts job experiences listed in [ESCO](https://esco.ec.europa.eu/en/about-esco) from raw CV texts, using SpaCy's `PhraseMatcher`.

* **SemanticMatcher**: once job experiences are extracted, the `SemanticMatcher` (`all-MiniLM-L6-v2`) is then used exclusively to determine which of these experiences match the parser extracted skills, employing semantic embeddings. Pairwise cosine similarity is calculated between the embeddings of **JobParser** skills and Parser skills. Matches are established when this similarity exceeds a specified threshold. This matching step is used only to compute metrics.


In [ ]:
parser = JobParser(normalized_jobs)
matcher = SemanticMatcher()

In [ ]:
res_job = compute_candidate_coverage(
    df_cv,
    df_skills_cleaned,
    "Job_title",
    parser.parse_with_n_grams,
    matcher=matcher.semantic_comparison,
)

print("Confusion matrix:", res_job.conf)

In [ ]:
df_fn = pl.DataFrame(res_job.fn_rows)
sample = df_fn.sample(n=2, shuffle=True)
for row in sample.to_dicts():
    print_highlighted_cv(row, pattern=jobs_pattern)

In [ ]:
print(f"False negatives matching snippet pattern: {df_fn.height}")
df_fn.write_csv(JOB_TITLE_FALSE_NEGATIVES_PATH, separator=";")
print("Saved filtered false negatives to false_negative.csv")

In [ ]:
%%bash --bg 
cd ..

# for Unix users
.venv/bin/python -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

# for Windows users
#.venv/Scripts/python.exe -m streamlit run hiring_cv_bias/bias_detection/rule_based/app/fn_app.py

In [ ]:
print_report(
    result=res_job,
    df_population=df_cv,
    reference_col="Male",
    group_col="Gender",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_job,
    df_population=df_cv,
    reference_col="NORTH",
    group_col="Location",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=res_job,
    df_population=df_cv,
    reference_col="LONG",
    group_col="length",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

### Bias detection per skill

In [ ]:
df_skills_cleaned = df_skills.with_columns(
    pl.col("Skill")
    .str.to_lowercase()
    .str.replace_all("(m/f)", "", literal=True)
    .str.strip_chars()
    .alias("Skill")
)

In [ ]:
parser = JobParser(normalized_jobs)
matcher = SemanticMatcher()

In [ ]:
res_job = compute_candidate_coverage(
    df_cv,
    df_skills_cleaned,
    "Job_title",
    parser.parse_with_n_grams,
    matcher=matcher.semantic_comparison,
)

print("Confusion matrix:", res_job.conf)

In [ ]:
candidate_ids = set(df_cv["CANDIDATE_ID"].to_list())
skills_result = extract_skill_cases(res_job, df_cv, {"baby-sitter", "babysitter"})

In [ ]:
print_report(
    result=skills_result,
    df_population=df_cv,
    reference_col="Female",
    group_col="Gender",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=skills_result,
    df_population=df_cv,
    reference_col="NORTH",
    group_col="Location",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
print_report(
    result=skills_result,
    df_population=df_cv,
    reference_col="LONG",
    group_col="length",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

In [ ]:
gender_jobs_df = load_data(GENDER_JOBS_PATH)
gender_jobs_df = gender_jobs_df.filter(pl.col("count_female") > pl.col("count_male"))
gender_jobs_df

In [ ]:
avg_perc_male = 0
avg_perc_female = 0

for jobs in gender_jobs_df.iter_rows(named=True):
    avg_perc_male += jobs["perc_male_zippia"] * jobs["count_total"]
    avg_perc_female += jobs["perc_female_zippia"] * jobs["count_total"]


avg_perc_male /= gender_jobs_df["count_total"].sum()
avg_perc_female /= gender_jobs_df["count_total"].sum()

In [ ]:
print(f"DI expected: {(avg_perc_male / avg_perc_female):.4f}")

In [ ]:
gender_jobs_df = gender_jobs_df.with_columns(
    pl.col("Skill")
    .str.to_lowercase()
    .str.replace_all("(m/f)", "", literal=True)
    .str.strip_chars()
    .alias("Skill")
)
gender_jobs = gender_jobs_df["Skill"].to_list()
gender_jobs

In [ ]:
esco_variants = []
for job in gender_jobs_df["Skill"].to_list():
    best_score = 0.0
    for esco_job in normalized_jobs:
        closest = ratio(job, esco_job)
        if closest > best_score:
            best_score = closest
            best_job = esco_job

    if best_score > 0.80:
        esco_variants.append(best_job)
    else:
        similarities = matcher.get_similarity(job, normalized_jobs)
        if max(similarities[0]) > 0.6:
            esco_variants.append(normalized_jobs[np.argmax(similarities.cpu())])

esco_variants

In [ ]:
candidate_ids = set(df_cv["CANDIDATE_ID"].to_list())
skills_result = extract_skill_cases(res_job, df_cv, set(esco_variants + gender_jobs))

In [ ]:
print_report(
    result=skills_result,
    df_population=df_cv,
    reference_col="Female",
    group_col="Gender",
    metrics=[
        "equality_of_opportunity",
        "calibration_npv",
    ],
)

**Final disparate_impact DI = DI_observed (0.07) / DI_expected (0.16) = 0.44. (<<0.80)**